# Intro to Spark
Ejemplos prácticos
---

Crear una sesión de Spark

In [ ]:
### Con SparkSession
from pyspark.sql import SparkSession

# crear sesión

spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("local[*]") \
        .getOrCreate()

In [ ]:
## parar la sesión
spark.stop()

In [ ]:
## Con SparkContext y SQLContext
from pyspark import SparkContext
from pyspark import SQLContext

## crear el contexto
sc = SparkContext("local[*]", "pyspark_df")
ss = SQLContext(sc)


In [ ]:
### parar el contexto
sc.stop()

---

In [ ]:
### Crear un Dataframe a partir de un csv
dfs_flights = spark.read.option("inferSchema", "true").option("header", "true").csv('data/2015-summary.csv')

In [ ]:
### Inspeccionar el Dataframe
dfs_flights.printSchema()

In [ ]:
## Renombrar la cplumna count

dfs_flights = dfs_flights.withColumnRenamed('count', 'NumFlights')

In [ ]:
dfs_flights.printSchema()

In [ ]:
### Ordenar según la columna count
dfs_fligths_sorted = dfs_flights.sort("NumFlights", ascending=False) # no tarda nada porque es una transformación

In [ ]:
dfs_fligths_sorted.show(10) ## al hacerle el show realiza el sort y nos devuelve los 10 primeros

In [ ]:
### group by
count_per_dest = dfs_flights.groupBy("DEST_COUNTRY_NAME").sum()

In [ ]:
### Enseña el dataframe count_per_dest
count_per_dest.show()

In [ ]:
## Renombra la columna sum(NumFlights)
count_per_dest = count_per_dest.withColumnRenamed('sum(NumFlights)', 'total_flights')
count_per_dest.show()

In [ ]:
### Ordena por número de vuelos
most_flights = count_per_dest.sort('total_flights', ascending=False)

In [ ]:
### Top 5
most_flights.show(5)

### Querying the data with SQL

In [ ]:
# Create temp table

df_flights.createOrReplaceTempView("flights_2015")

In [ ]:
query = """
SELECT *
FROM flights_2015
ORDER BY count DESC
LIMIT 10
"""

In [ ]:
query_result = spark.sql(query)

In [ ]:
type(query_result)

In [ ]:
query_result.show()

In [ ]:
# Calcula el total de vuelos en función del destino y saca el destino más frecuente

## Funcionalidad dataframes

Muchas veces nos encontraremos con que queremos utilizar algua funcion integrada en los dataframes, cómo en pandas.

Para poder acceder a estas funciones, tendremos que importarlas de la librería

In [ ]:
from pyspark.sql.functions import max

In [ ]:
spark.sql("SELECT max(count) from flights_2015").take(1)

In [ ]:
dfs_flights.select(max("NumFlights")).take(1)

### Intefración con pandas

In [ ]:
import pandas as pd

In [ ]:
df = dfs_flights.toPandas()

In [ ]:
df.head()

In [ ]:
df = dfs_flights.groupBy("DEST_COUNTRY_NAME").sum("NumFlights")\
           .withColumnRenamed("sum(NumFlights)", "destination_total")\
           .sort(desc("destination_total"))\
           .limit(10)\
           .toPandas()

In [ ]:
%matplotlib inline

In [ ]:
df.set_index('DEST_COUNTRY_NAME').plot(kind='barh')

In [ ]:
## from pandas to SparkDataframe

df_test = spark.createDataFrame(df)

In [ ]:
spark.stop()

## Ejercicio
---

Haz un análisis exploratorio del fichero 'data/201508_trip_data.csv' usando spark y plotea los resultados usando las funcionalidades de python

In [ ]:
## importa lo necesario para crear el entorno de spark

### Con SparkSession
from pyspark.sql import SparkSession

# crear sesión

spark = SparkSession.builder.appName("Bikes").config("local[*]").getOrCreate()

In [ ]:
## carga el fichero 'data/201508_trip_data.csv'
path = 'data/201508_trip_data.csv'

dfs_bikes = spark.read.option('inferSchema', True).option('header', True).csv(path)

In [ ]:
# Imprime el esquema y las columnas del dfs_bikes

dfs_bikes.printSchema()

In [ ]:
## Haz una descripción de la columnas Duration del dfs_bikes
## En que unidades está la duración ?
dfs_bikes.select('Duration').describe().show()

In [ ]:
## Cual son las 5 estaciones más transitada?

most_frequent = dfs_bikes.select('Zip Code', 'Duration').groupby('Zip Code').count()\
                .withColumnRenamed('count', 'num_trips')\
                .sort('num_trips', ascending=False).limit(5)

In [ ]:
## Repite el ejercicio anterior usando SQL

## Cambia el nombre a la columna para que no tenga espacios
dfs_bikes = dfs_bikes.withColumnRenamed('Zip Code', 'zipcode')

## Crea una tabla temporal
dfs_bikes.createOrReplaceTempView('bike_trips')

## Escribe una query sql que realice la tarea de antes
query = """

SELECT zipcode, count(1) as num_trips
FROM bike_trips
GROUP BY zipcode
ORDER BY num_trips DESC
LIMIT 5
"""

most_frequent_sql = spark.sql(query)

In [ ]:
### Transform it to pandas and plot the result
